In [7]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jan 30 21:47:59 2019

@author: mwern
"""

# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""

import pandas as pd
import numpy as np
import pandas_datareader as web
import datetime
from datetime import timedelta
import random
import winsound
import csv
from twilio.rest import Client
print(datetime.datetime.now())

pd.set_option('display.max_columns',500)

start = datetime.date(2018,1,1)
end = datetime.date.today()
#end = datetime.date(2019,2,11)

sp500 = web.get_data_yahoo('^GSPC', start=start, end=end)   
sp500["10d"] = np.round(sp500["Close"].rolling(window = 10, center= False).mean(),2)
sp500["30d"] = np.round(sp500["Close"].rolling(window = 30, center= False).mean(),2)
sp500 = sp500.reset_index()  

columns = ['Date','High', 'Low', 'Open','Close', 'Volume', 'Adj Close', 'Symbol','ATR','TenSMA','ThirtySMA', 'TenSMALast', 'ThirtySMALast','Avg Vol', '52w', 'DailyReturn', 'Days', 'DaysLast'] 
tradecol = ['Date','Symbol', 'Shares', 'BuyPrice', 'SellPrice', 'SellDate', 'Profit', 'Stage'] 
trades = pd.DataFrame(columns=tradecol)  
buylist = pd.DataFrame(columns=columns)
buylist52 = pd.DataFrame(columns=columns)
master = pd.DataFrame(columns=columns)
#master = pd.read_csv('master.csv', index_col=0) 

with open('MasterStockListSP500.csv', 'r') as f:
        reader = csv.reader(f)
        symbols = list(reader)
#symbols = random.sample(symbols, 300)
sizeofsymbols = len(symbols)

def populatemaster(master) :
    l = 0
    while l < sizeofsymbols :
        s = symbols[l]
        s = str(symbols[l])
        s = s.replace('[', '')
        s = s.replace(']', '')
        s = s.replace("'", '')
        #print(s)
        try:
            stock = web.get_data_yahoo(s, start=start, end=end)
        except: pass
        stock['ATR1'] = abs(stock['High'] - stock['Low'])
        stock['ATR2'] = abs(stock['High'] - stock['Close'].shift())
        stock['ATR3'] = abs(stock['Low'] - stock['Close'].shift())
        stock["TR"] = stock[['ATR1', 'ATR2', 'ATR3']].max(axis=1)
        stock["Symbol"] = s
        stock["ATR"] = np.round(stock["TR"].rolling(window = 14, center= False).mean(),2)
        stock['ATR'] = (stock['ATR'].shift(1)*13 + stock['TR'] ) /  14
        stock["TenSMA"] = np.round(stock["Close"].rolling(window = 10, center= False).mean(),2)
        stock["ThirtySMA"] = np.round(stock["Close"].rolling(window = 30, center= False).mean(),2)
        stock["TenSMALast"] = stock.TenSMA.shift(1)
        stock["ThirtySMALast"] = stock.ThirtySMA.shift(1)
        stock["AvgVol"] = np.round(stock["Volume"].rolling(window = 30, center= False).mean(),2)
        stock["52w"] = np.round(stock["High"].rolling(window = 250, center= False).max(),2)
        stock['DailyReturn'] = (stock['Close'] - stock.Close.shift(1))/stock.Close.shift(1)
        rmlag = lambda xs: np.argmax(xs[::-1])
        stock["Days"] = stock["High"].rolling(window=250).apply(func=rmlag, raw=True)
        stock["DaysLast"] = stock.Days.shift(1)
        stock = stock.reset_index()
        #masteradd = pd.DataFrame([stock], columns=columns)    
        master = master.append([stock], ignore_index = True, sort=False)
        l = l + 1
    
    return(master)        
date = datetime.date.today()
#date = datetime.date(2019,7,26)
master = populatemaster(master)
buylist = master.loc[(master.TenSMA > master.ThirtySMA) & (master.TenSMALast < master.ThirtySMALast) & (master.Date == date.strftime("%Y/%m/%d"))]
buylist52 = master.loc[(master.Days == 0.0) & (master.DaysLast > 20.0) & (master.Date == date.strftime("%Y/%m/%d")) ]
buylist = buylist.reset_index()
buylist52 = buylist52.reset_index()
buylistSMS = buylist['Symbol'].tolist()

#t30 = master.loc[(master.Low < master['30d']) & (master.High > master['30d']) & (0.99*master.Open < master.Close) & (master.Close < 1.01*master.Open) & (master['10d'] > master['30d']) & (master.TR > master.ATR) & (master.Close > .995 * master.High) & (master.Low < 0.97 * master.High) & (master.Volume > master.Volume.shift(1)) & (master.Date == date.strftime("%Y/%m/%d"))]
#print(t30)
# Your Account Sid and Auth Token from twilio.com/console
account_sid = 'AC3db1d26b5045dc8457971911f613ae85'
auth_token = '3c57085b577a7317dcf5d902e54e8bce'
client = Client(account_sid, auth_token)

message = client.messages \
                .create(
                    body = str(buylistSMS),
                    from_='+13026001582',
                     to='+14848430716'
                 )

#print(message.sid)
print(buylist)
print(buylist52)


#buylist = master.loc[np.logical_and(master['Date'] == date.strftime("%Y/%m/%d"), master.Symbol == portfolio.iloc[e]['Symbol'])]
duration = 1000
freq = 440
winsound.Beep(freq, duration)
print(datetime.datetime.now())
#l = 0
#while l < sizeofsymbols :
#     s = symbols[l]
#     s = str(symbols[l])
#     s = s.replace('[', '')
#     s = s.replace(']', '')
#     s = s.replace("'", '')
#     #print(s)
#     stock = web.get_data_yahoo(s, start=start, end=end)
#     stock['ATR1'] = abs(stock['High'] - stock['Low'])
#     stock['ATR2'] = abs(stock['High'] - stock['Close'].shift())
#     stock['ATR3'] = abs(stock['Low'] - stock['Close'].shift())
#     stock["TR"] = stock[['ATR1', 'ATR2', 'ATR3']].max(axis=1)
#     stock["Symbol"] = s
#     stock["ATR"] = np.round(stock["TR"].rolling(window = 14, center= False).mean(),2)
#     stock['ATR'] = (stock['ATR'].shift(1)*13 + stock['TR'] ) /  14
#     stock["10d"] = np.round(stock["Close"].rolling(window = 10, center= False).mean(),2)
#     stock["30d"] = np.round(stock["Close"].rolling(window = 30, center= False).mean(),2)
#     stock["AvgVol"] = np.round(stock["Volume"].rolling(window = 30, center= False).mean(),2)
#     stock["52w"] = np.round(stock["High"].rolling(window = 250, center= False).max(),2)
#     rmlag = lambda xs: np.argmax(xs[::-1])
#     stock["Days"] = stock["High"].rolling(window=250).apply(func=rmlag, raw=True)
#     stock["DaysLast"] = stock.Days.shift(1)
#     stock = stock.reset_index()
#        
#     today = stock.index[-1]
#     #if stock.loc[today]['Days'] == 0.0 and stock.loc[today]['DaysLast'] > 20.0 and stock.iloc[today]['AvgVol'] > 300000 and stock.loc[today]['Close'] > 15.0  and stock.loc[today]['10d'] < stock.loc[today]['30d'] :
#     buylist.loc[stock.index[today]] = stock.iloc[today]
#         
#     l += 1

#
#     stage = "Waiting"
# 
# 
#     for i, row in stock.iterrows():
#         if stage == "Waiting" :
#             if stock.loc[i]['Days'] == 0.0 and stock.loc[i]['DaysLast'] > 25.0 and stock.iloc[i]['AvgVol'] > 300000 and stock.loc[i]['Close'] > 15.0  and stock.loc[i]['10d'] > stock.loc[i]['30d'] :
#                 buydate = stock.loc[i]['Date']
#                 buyprice = stock.loc[i]['Close']
#                 #stop = stock.loc[i]['Close'] * (1 - initialstop)
#                 stop = stock.loc[i]['Close'] - 3*stock.loc[i]['ATR']
#                 #target = stock.loc[i]['Close'] * 1.05
#                 shares = r/(3*stock.loc[i]['ATR'])
#                 stage = "Initial Stop"            
#         elif stage == "Initial Stop" :
#             if stock.loc[i]['Low'] < stop :
#                 sellprice = stop
#                 selldate = stock.loc[i]['Date']
#                 stage = "Waiting"
#                 profit = (shares*(sellprice - buyprice)) - 10
#                 data = [buydate, s, shares, buyprice, sellprice, selldate, profit, stage] 
#                 trades2 = pd.DataFrame([data], columns=tradecol)    
#                 trades = trades.append(trades2, ignore_index = True) 
#             #elif stock.loc[i]['Close'] > target :
#             else :
#                 #stop = max(stop, stock.loc[i]['Close'] * (1-initialstop))
#                 stop = max(stop, stock.loc[i]['Close'] - 3*stock.loc[i]['ATR'])
#                 stage = "Trailing Stop"
#         else :
#             if stock.loc[i]['Low'] < stop :
#                 sellprice = stop
#                 selldate = stock.loc[i]['Date']
#                 stage = "Waiting"
#                 profit = (shares*(sellprice - buyprice)) - 10
#                 data = [buydate, s, shares, buyprice, sellprice, selldate, profit, stage] 
#                 trades2 = pd.DataFrame([data], columns=tradecol)    
#                 trades = trades.append(trades2, ignore_index = True) 
#             else :
#                 #stop = max(stop, stock.loc[i]['Close'] * (1-initialstop))
#                 stop = max(stop, stock.loc[i]['Close'] - 3*stock.loc[i]['ATR'])
#                 if i == len(stock) - 1:
#                     sellprice = stock.loc[i]['Close']
#                     profit = (shares*(sellprice - buyprice)) - 10
#                     selldate = stock.loc[i]['Date']
#                     data = [buydate, s, shares, buyprice, sellprice, selldate, profit, stage] 
#                     trades2 = pd.DataFrame([data], columns=tradecol)    
#                     trades = trades.append(trades2, ignore_index = True)
#     l += 1
# 
# 
#trades.loc['Total'] = trades.sum()
#print(trades)

        

     
     




2019-09-10 21:54:39.613456
    level_0       Date         High          Low         Open        Close  \
0      1274 2019-09-10   155.380005   146.789993   147.369995   154.699997   
1      5099 2019-09-10    40.349998    39.689999    39.689999    40.290001   
2     11899 2019-09-10    65.480003    64.510002    65.010002    64.989998   
3     12324 2019-09-10   106.019997   103.940002   105.989998   105.680000   
4     14449 2019-09-10    90.000000    87.720001    90.000000    88.400002   
5     15724 2019-09-10   144.630005   139.029999   139.149994   144.630005   
6     19124 2019-09-10    24.450001    23.520000    23.900000    23.700001   
7     26329 2019-09-10    51.270000    49.689999    50.639999    51.240002   
8     28029 2019-09-10    39.320000    37.939999    38.049999    39.099998   
9     32279 2019-09-10    38.459999    36.490002    36.540001    38.410000   
10    32704 2019-09-10   130.910004   129.050003   129.669998   130.910004   
11    33554 2019-09-10    29.889999  

2019-09-10 22:02:28.165570


# jjjjj

In [4]:
buylist52


,level_0,Date,High,Low,Open,Close,Volume,Adj Close,Symbol,ATR,TenSMA,ThirtySMA,TenSMALast,ThirtySMALast,Avg Vol,52w,DailyReturn,Days,DaysLast,ATR1,ATR2,ATR3,TR,AvgVol,index
0,6719,2019-09-03,78.279999,76.959999,77.220001,78.205002,792196.0,78.205002,AEE,1.069286,76.56,76.28,76.44,76.21,NaN,78.28,0.013675,0.0,48.0,1.320000,1.129997,0.190002,1.320000,1894409.87,NaN
1,7139,2019-09-03,92.750000,90.989998,91.220001,92.559998,883556.0,92.559998,AEP,1.397857,90.82,89.86,90.62,89.78,NaN,92.75,0.015469,0.0,48.0,1.760002,1.599998,0.160004,1.760002,2204968.53,NaN
2,22243,2019-09-03,214.490005,212.000000,211.919998,213.589996,310561.0,213.589996,AVB,2.861429,209.39,207.05,208.80,206.77,NaN,214.49,0.004846,0.0,37.0,2.490005,1.930008,0.559998,2.490005,596368.70,NaN
3,100186,2019-09-03,66.400002,65.050003,65.050003,65.690002,923881.0,65.690002,INFO,1.155000,64.68,64.42,64.65,64.41,NaN,66.40,0.001219,0.0,30.0,1.349998,0.790001,0.559998,1.349998,1616419.37,NaN
4,114466,2019-09-03,127.099998,124.779999,124.879997,125.720001,627815.0,125.720001,KSU,2.728571,123.02,121.44,122.66,121.43,NaN,127.10,-0.000636,0.0,28.0,2.320000,1.299995,1.020004,2.320000,740567.17,NaN
5,150821,2019-09-03,34.029999,33.549999,33.549999,33.974998,1519226.0,33.974998,PHM,0.637857,33.29,32.21,33.14,32.19,NaN,34.03,0.005177,0.0,29.0,0.480000,0.230000,0.250000,0.480000,3402470.87,NaN
6,183529,2019-09-03,43.759998,41.639999,42.380001,41.803001,12422802.0,41.803001,TWTR,1.228571,42.05,41.29,42.04,41.15,NaN,43.76,-0.019859,0.0,21.0,2.119999,1.109997,1.010002,2.119999,15471043.40,NaN


In [6]:
buylist

,level_0,Date,High,Low,Open,Close,Volume,Adj Close,Symbol,ATR,TenSMA,ThirtySMA,TenSMALast,ThirtySMALast,Avg Vol,52w,DailyReturn,Days,DaysLast,ATR1,ATR2,ATR3,TR,AvgVol,index
0,24515,2019-09-06,1154.150024,1138.489990,1141.650024,1145.349976,133377.0,1145.349976,AZO,22.033570,1101.56,1097.29,1095.18,1097.31,NaN,1186.60,0.009528,37.0,36.0,15.660034,19.609985,3.949951,19.609985,255209.23,NaN
1,52010,2019-09-06,95.160004,94.269997,94.510002,94.839996,860622.0,94.839996,CTXS,1.205715,92.92,92.85,92.73,92.86,NaN,112.87,0.005193,246.0,245.0,0.890007,0.810005,0.080002,0.890007,2129320.73,NaN
2,56663,2019-09-06,144.899994,143.115005,143.589996,144.600006,1070796.0,144.600006,DHR,2.748571,140.10,139.88,139.59,139.83,NaN,145.50,0.011543,44.0,43.0,1.784988,1.949997,0.165009,1.949997,1904173.20,NaN
3,58355,2019-09-06,35.889999,34.720001,34.930000,35.770000,1318382.0,35.770000,DISH,0.993571,33.40,33.14,33.00,33.25,NaN,44.66,0.026399,31.0,30.0,1.169998,1.040001,0.129997,1.169998,2872739.40,NaN
4,69776,2019-09-06,83.480003,81.879997,81.980003,83.379997,1319176.0,83.379997,ETN,1.776429,79.15,79.05,78.65,79.00,NaN,89.46,0.017450,240.0,239.0,1.600006,1.530006,0.070000,1.600006,2457119.20,NaN
5,94559,2019-09-06,14.610000,14.390000,14.470000,14.590000,5773867.0,14.590000,HPE,0.396429,13.57,13.49,13.44,13.48,NaN,17.27,0.013194,240.0,239.0,0.219999,0.210000,0.009999,0.219999,9697188.90,NaN
6,118247,2019-09-06,27.219999,26.850000,27.000000,26.860001,1217056.0,26.860001,LKQ,0.695000,26.02,25.93,25.92,25.96,NaN,33.53,-0.001858,249.0,249.0,0.369999,0.309999,0.059999,0.369999,2527388.53,NaN
7,118670,2019-09-06,27.219999,26.850000,27.000000,26.860001,1217056.0,26.860001,LLL,0.695000,26.02,25.93,25.92,25.96,NaN,33.53,-0.001858,249.0,249.0,0.369999,0.309999,0.059999,0.369999,2527388.53,422.0
8,130091,2019-09-06,101.714996,100.900002,100.750000,101.032898,935143.0,101.032898,MMC,1.413928,98.79,98.63,98.50,98.65,NaN,103.37,-0.000169,28.0,27.0,0.814995,0.664993,0.150002,0.814995,1506034.77,NaN
9,162855,2019-09-06,80.989998,79.550003,80.190002,80.677299,461891.0,80.677299,RJF,1.765000,77.70,77.56,77.49,77.64,NaN,96.76,0.009097,241.0,240.0,1.439995,1.040001,0.399994,1.439995,879749.70,NaN
